In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.offline import plot
from plotly.utils import PlotlyJSONEncoder
import json


# 加载数据
df_RPKM_filter = pd.read_csv('demo_RPKM.txt',sep='\t').copy()
df_RPKM = pd.read_csv('demo_RPKM2.txt',sep='\t').copy()

In [3]:
def heatmap(df, corlor, width=900, height=600, button='Z-score'):
    '''根据输入的表达矩阵，绘制热图
    Args:
        df: 表达矩阵
        corlor: 颜色渐变方式
        button: 标准化方法
    Returns:
        fig: 热图
    '''

    # df第一列改名为gene_id
    df = df.rename(columns={df.columns[0]: 'gene_id'})
    # 提取gene_id作为索引
    df = df.set_index('gene_id')

    if button == 'Z-score':
        # 将数据进行归一化，范围-2到2
        df_normalized = df.apply(lambda x: np.interp(x, (x.min(), x.max()), (-2, 2)))
    
    # 如果button==log2，那么进行log2变换
    elif button == 'log2':
        df_normalized = np.log(df+1)
    else:
        df_normalized = df

    # 将归一化后的DataFrame转置，使样本名称在x轴，基因id在y轴
    df_normalized = df_normalized.transpose()

    # 创建热图
    fig = px.imshow(df_normalized,
                    labels=dict(x="Sample", y="Gene ID", color="Exp"),
                    x=df_normalized.columns,
                    y=df_normalized.index,
                    color_continuous_scale=corlor
                   )

    # 定义鼠标悬浮数据显示方式
    fig.update_traces(hovertemplate='Gene: %{x}<br>Sample: %{y}<br>RPKM: %{z:.1f}')
    fig.update_layout(title='Gene expression heatmap')

    # 设置输出图的大小
    fig.update_layout(width=width, height=height)

    # 将fig转为json格式，返回json数据
    # fig_json = json.dumps(fig, cls=PlotlyJSONEncoder)
    # return fig_json

    # 将fig转为html格式，返回html代码
    # plot_html = plot(fig, output_type='div', include_plotlyjs=False)        
    # return plot_html

    return fig

# 调用函数
heatmap(df_RPKM_filter, 'Rdbu_r', 1000, 600, 'Z-score')

In [5]:
# 调用函数
heatmap(df_RPKM_filter, 'Rdbu_r', 1000, 600, 'log2')

In [7]:
# 调用函数
heatmap(df_RPKM_filter, 'Rdbu_r', 1000, 600, '')

In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def create_heatmap(df):
    # df第一列改名为gene_id
    df = df.rename(columns={df.columns[0]: 'gene_id'})
    # 提取gene_id作为索引
    df = df.set_index('gene_id')
    # 将数据进行归一化，范围-2到2
    df_normalized = df.apply(lambda x: np.interp(x, (x.min(), x.max()), (-2, 2)))
    # 创建热图
    fig = go.Figure(data=go.Heatmap(
        z=df_normalized,
        x=df_normalized.columns,
        y=df_normalized.index,
        colorscale='RdBu_r',
        hovertemplate='Sample: %{x}<br>Gene ID: %{y}<br>FPKM: %{z:.3f}<extra></extra>'
    ))
    # 设置轴标签
    fig.update_layout(
        xaxis_title="Sample",
        yaxis_title="Gene ID"
    )
    return fig

create_heatmap(df_demo_fpkm)